In [8]:
import sqlite3
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [9]:
# Create or connect to SQLite DB
conn = sqlite3.connect("sample.db")
cursor = conn.cursor()

In [ ]:
# Create the required table columns and enter data of each rows..

cursor.execute("""
CREATE TABLE IF NOT EXISTS customers (
    id INTEGER PRIMARY KEY,
    name TEXT,
    total_purchase REAL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    amount REAL,
    order_date TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
)
""")

cursor.executemany("INSERT INTO customers (id, name, total_purchase) VALUES (?, ?, ?)", [
    (1, "Alice", 7500),
    (2, "Bob", 4500),
    (3, "Charlie", 5200)
])

cursor.executemany("INSERT INTO orders (id, customer_id, amount, order_date) VALUES (?, ?, ?, ?)", [
    (1, 1, 3000, "2024-01-01"),
    (2, 1, 4500, "2024-02-10"),
    (3, 2, 4500, "2024-03-01"),
    (4, 3, 5200, "2024-03-15")
])

In [ ]:
# To get table schema
cursor.execute("PRAGMA table_info(customers)")
schema_rows = cursor.fetchall()

In [21]:
schema_rows

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'total_purchase', 'REAL', 0, None, 0)]

In [13]:
# Format schema as text
schema_text = "Table customers with columns: " + ", ".join(
    f"{col[1]} ({col[2]})" for col in schema_rows
)

In [22]:
schema_text

'Table customers with columns: id (INTEGER), name (TEXT), total_purchase (REAL)'

In [ ]:
# Load tokenizer and model
model_name = "cssupport/t5-small-awesome-text-to-sql"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [16]:
# Natural language input
question = "How many customers have spent more than 5000?"

# Final prompt to model
input_text = f"{schema_text}. Question: {question}"
input_ids = tokenizer.encode(f"translate English to SQL: {input_text}", return_tensors="pt")

In [24]:
print(input_text)
print(input_ids)

Table customers with columns: id (INTEGER), name (TEXT), total_purchase (REAL). Question: How many customers have spent more than 5000?
tensor([[13959,  1566,    12, 12558,    10,  4398,   722,    28, 15752,    10,
             3,    23,    26,    41, 21342, 17966,   201,   564,    41,  3463,
             4,   382,   201,   792,   834, 29446,    41, 13223,   434,   137,
         11860,    10,   571,   186,   722,    43,  1869,    72,   145,     3,
         12814,    58,     1]], device='cuda:0')


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)  # Move model to GPU or CPU
input_ids = input_ids.to(device)  # Move input to same device

In [25]:
device

device(type='cuda')

In [19]:
# Generate SQL
outputs = model.generate(input_ids, max_length=64)
sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated SQL:", sql_query)

Generated SQL: SELECT COUNT(*) FROM customers WHERE total_purchase > 5000


In [26]:
outputs

tensor([[    0,     3, 23143, 14196,  2847, 17161,   599,  1935,    61, 21680,
           722,   549, 17444,   427,   792,   834, 29446,  2490,     3, 12814,
             1]], device='cuda:0')

In [20]:
try:
    cursor.execute(sql_query)
    result = cursor.fetchall()
    print("SQL Result:", result)
except Exception as e:
    print("Error executing SQL:", e)

conn.close()

SQL Result: [(2,)]
